In [5]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from palettable.scientific.sequential import Batlow_5
from palettable.tableau import Tableau_10
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

print(os.environ['SUBGRAPH_URL'])
# logging.basicConfig(level=logging.DEBUG)

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    chart_bin_count_value_aggregate
)
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

https://graph.node.bean.money/subgraphs/name/beanstalk


In [6]:
ADDR_BEANSTALK.lower()

'0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5'

In [7]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [8]:
farmers = bs.Query.farmers(
    first=100000, 
    where= {'silo_': {'id_not': "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5"}}
)
df_farmers = sg.query_df(
    [
        farmers.id, 
        farmers.deposits.bdv,  
    ], 
    pagination_strategy=ShallowStrategy
)
df_farmers = remove_prefix(df_farmers, "farmers_")
df_farmers = remove_prefix(df_farmers, "deposits_")

In [9]:
df_farmers.bdv = df_farmers.bdv / 1e6 
df_farmers = df_farmers.rename(columns={"id": "address"})
assert df_farmers.bdv.min() >= 0
df_farmers.head()

,address,bdv
0,0x0000000002ce79aacd54227d2163ff3791338975,0.000000
1,0x0000002e4f99cb1e699042699b91623b1334d2f7,0.000000
2,0x0000002e4f99cb1e699042699b91623b1334d2f7,0.000000
3,0x0000002e4f99cb1e699042699b91623b1334d2f7,0.000167
4,0x0000002e4f99cb1e699042699b91623b1334d2f7,0.000000


In [10]:
breakpoints = [
    1, 1e3, .5e4, 1e4, 2.5e4, 5e4, 1e5, 2.5e5, 5e5, 1e6, 5e6, float("inf") 
]
c = chart_bin_count_value_aggregate(df_farmers, "bdv", breakpoints)
c

alt.VConcatChart(...)

In [11]:
output_chart(c)

<IPython.core.display.JSON object>